## Prepare the data for the model ##

The processed data used in the paper is available on the github repo. However, detailed information on steps required to process the data is not. These pre-processing notebooks aim to create a dataset that can be easily used by the model.

We take the following steps:

<img src="../../images/data_path.png" alt="Data Path" style="width:60%;">

Demean, detrend, and resample are done in the [Data_Cleaning](Data_Cleaning.ipynb) notebook.



Read the files and segment them into one hour windows. Each file will contain 28,800 samples per hour at an 8 Hz sampling rate.

In [14]:
import os
from obspy import read
import glob
import numpy as np
from obspy import UTCDateTime

# Define the processed and output folder paths
processed_folder = os.getcwd() + '/data/processed'
segmented_output_folder = os.getcwd() + '/data/segmented'
os.makedirs(segmented_output_folder, exist_ok=True)  # Create output directory if it doesn't exist

# Define the target segment duration (1 hour) in seconds
segment_duration = 3600  # seconds

# Process each processed file for segmentation
for file_path in glob.glob(f"{processed_folder}/*.mseed"):
    # Read the processed file
    st = read(file_path)
    
    # Split each trace in the Stream object into one-hour segments
    for tr in st:
        start_time = tr.stats.starttime
        end_time = tr.stats.endtime
        segment_start = start_time
        
        # Loop over each one-hour segment
        while segment_start + segment_duration <= end_time:
            # Define the end time for the current segment
            segment_end = segment_start + segment_duration

            # Slice the trace to create a one-hour segment
            segment = tr.slice(starttime=segment_start, endtime=segment_end)
            
            # Format the filename for the segment
            segment_filename = f"{tr.stats.network}_{tr.stats.station}_{tr.stats.channel}_{segment_start.strftime('%Y%m%dT%H%M%S')}.mseed"
            segment_filepath = os.path.join(segmented_output_folder, segment_filename)
            
            # Save the one-hour segment as a new file
            try:
                segment.write(segment_filepath, format="MSEED")
                print(f"Saved segment file: {segment_filename}")
            except Exception as e:
                print(f"Error writing segment {segment_filename}: {e}")

            # Move to the next hour
            segment_start = segment_end

Saved segment file: 9F_NUPH_HHE_20210424T224800.mseed
Saved segment file: 9F_NUPH_HHE_20210424T234800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T004800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T014800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T024800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T034800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T044800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T054800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T064800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T074800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T084800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T094800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T104800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T114800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T124800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T134800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T144800.mseed
Saved segment file: 9F_NUPH_HHE_20210425T154800.mseed
Saved segment file: 9F_NUPH_

In [15]:
# check all files in the segmented folder for nan data

for file_path in glob.glob(f"{segmented_output_folder}/*.mseed"):
    st = read(file_path)
    for tr in st:
        if np.isnan(tr.data).any():
            print(f"File {file_path} contains NaN data")
        # else:
        #     print(f"File {file_path} contains no NaN data")


Apply the Fourier Transform with the `specified window_length` and `hop_length parameters` to produce a spectrogram of size (96, 128) for each one-hour segment. The data is saved in `data\segmented`.

In [16]:
import os
import glob
import numpy as np
from obspy import read
import librosa
from scipy.signal import resample
import time
from IPython.display import display, clear_output
import torch
from scipy.ndimage import zoom

# Define directories
transformed_output_folder = os.getcwd() + '/data/transformed'
os.makedirs(transformed_output_folder, exist_ok=True)  # Create output directory if it doesn't exist

# Define constants
target_sampling_rate = 8  # Hz
segment_duration = 3600  # 1 hour in seconds
window_length = 256
hop_length = 224
target_shape = (96, 128)

# Initialize an empty list to store spectrograms
spectrogram_list = []

# Step 1: Process each file in the input folder
for file_path in glob.glob(f"{segmented_output_folder}/*.mseed"):
    # Read the file
    st = read(file_path)
    #st.plot()
    
    # Process each trace
    for tr in st:
        #tr.plot()
       
        # Split into one-hour segments (28,800 samples each)
        start_time = tr.stats.starttime
        end_time = tr.stats.endtime
        segment_start = start_time

        while segment_start + segment_duration <= end_time:
            # Define the end time for the current segment
            segment_end = segment_start + segment_duration

            # Slice the trace to create a one-hour segment
            segment = tr.slice(starttime=segment_start, endtime=segment_end)

            # min max values in the segment
            #print(f"Min: {np.min(segment.data)}, Max: {np.max(segment.data)}")

            # Convert to PyTorch tensor
            segment_tensor = torch.tensor(segment, dtype=torch.float32)

            print(f"Min: {torch.min(segment_tensor)}, Max: {torch.max(segment_tensor)}")
            
            # Ensure correct dimensions (STFT expects 1D tensor or batch x time)
            if len(segment_tensor.shape) == 1:
                segment_tensor = segment_tensor.unsqueeze(0)
                
            # Compute the STFT
            stft_result = torch.stft(
                segment_tensor,
                n_fft=window_length,
                hop_length=hop_length,
                win_length=window_length,
                return_complex=True,
            )
                
            # Compute the spectrogram (magnitude squared)
            spectrogram = torch.abs(stft_result) ** 2

            # Convert to NumPy for further processing
            spectrogram = spectrogram.numpy()

            print("converted")
            print(f'Min: {np.min(spectrogram)}, Max: {np.max(spectrogram)}')
            
            # Resize to a consistent shape if needed
            target_shape = (96, 128)  # Desired shape
            spectrogram_resized = np.resize(spectrogram, target_shape)
            print("resize")
            print(f'Min: {np.min(spectrogram_resized)}, Max: {np.max(spectrogram_resized)}')

            # Clip spectrogram values before log transformation NEWNEW
            #spectrogram_resized = np.clip(spectrogram_resized, a_min=0, a_max=1e6)  # Adjust `a_max` as needed
            print(" not clipped")
            #print(f'Min: {np.min(spectrogram_resized)}, Max: {np.max(spectrogram_resized)}')

            # Log-transform the spectrogram
            spectrogram_log = np.log1p(spectrogram_resized)
            print("log transformed")
            print(f'Min: {np.min(spectrogram_log)}, Max: {np.max(spectrogram_log)}')

            # Normalize between -1 and 1
            #spectrogram_log = 2 * (spectrogram_log - np.min(spectrogram_log)) / np.ptp(spectrogram_log) - 1
            
            #segment.plot()

            # Save the spectrogram (you could save the result as needed)
            segment_filename = f"{tr.stats.network}_{tr.stats.station}_{tr.stats.channel}_{segment_start.strftime('%Y%m%dT%H%M%S')}.npy"
            segment_filepath = os.path.join(transformed_output_folder, segment_filename)

            print(f'Min: {np.min(spectrogram_log)}, Max: {np.max(spectrogram_log)}\n')

            np.save(segment_filepath, spectrogram_log)

            # Append the spectrogram to the list
            spectrogram_list.append(spectrogram_log)
            
            # check the file for data outside the bounds of -1 and 1
            # if np.any(spectrogram_log > 1) or np.any(spectrogram_log < -1):
            #     print(f"File {segment_filename} contains data outside the bounds of -1 and 1")

            # Move to the next hour
            segment_start = segment_end
            
            # display file being processed
            # time.sleep(0.1)
            # clear_output(wait=True)
            print(f'Processing file: {segment_filename}')
            
            #check file for nan data
            if np.isnan(spectrogram_log).any():
                print(f"File {segment_filename} contains NaN data")             

# Stack all spectrograms into a single numpy array and add batch and channel dimensions
all_spectrograms = np.stack(spectrogram_list)
all_spectrograms = all_spectrograms[:, np.newaxis, :, :]  # Shape: (batch_size, 1, 96, 128)
print(f"Shape of combined spectrogram array: {all_spectrograms.shape}")

# remove the single feature dimension (1)
all_spectrograms = np.squeeze(all_spectrograms)

# NOTE: FILE NAME IS HARD CODED HERE
# Save the combined spectrogram array to Input.npy
final_output_folder = os.getcwd()
input_filepath = os.path.join(final_output_folder, "Input.npy")
#input_filepath = os.path.join(final_output_folder + 'NUPH_analysis', "Input_nuph.npy")
np.save(input_filepath, all_spectrograms)

print(f"Saved combined spectrogram file: {input_filepath.split('/')[-1]}")


Min: 0.012323705479502678, Max: 0.14658263325691223
converted
Min: 4.330104275140911e-09, Max: 892.59814453125
resize
Min: 4.8019973775126346e-08, Max: 892.59814453125
 not clipped
log transformed
Min: 4.8019973775126346e-08, Max: 6.7952561378479
Min: 4.8019973775126346e-08, Max: 6.7952561378479

Processing file: 9F_NUPH_HHE_20210331T174800.npy
Min: -0.0793299600481987, Max: 0.10050901025533676
converted
Min: 2.0137942424724997e-09, Max: 252.4638671875
resize
Min: 7.0619762482238e-08, Max: 252.4638671875
 not clipped
log transformed
Min: 7.0619762482238e-08, Max: 5.535221099853516
Min: 7.0619762482238e-08, Max: 5.535221099853516

Processing file: 9F_NUPH_HHE_20210413T054800.npy
Min: -0.2026721090078354, Max: 0.16453143954277039
converted
Min: 7.834591997379903e-08, Max: 1046.7783203125
resize
Min: 2.7225217991144746e-07, Max: 1046.7783203125
 not clipped
log transformed
Min: 2.72252151489738e-07, Max: 6.954427242279053
Min: 2.72252151489738e-07, Max: 6.954427242279053

Processing file:

In [17]:
# display information on Input.npy
print(f"Shape of combined spectrogram array with batch and channel dimensions: {all_spectrograms.shape}")
print(f"Number of files processed: {all_spectrograms.shape[0]}")

# load Input.npy and display information for confirmation
input_data_file_t = np.load(input_filepath)
print(f"Shape of combined spectrogram array Input.npy: {input_data_file_t.shape}")
print(f"Number of files processed: {input_data_file_t.shape[0]}")


# display max and min values of the Input.npy file
print(f"Max value in Input.npy: {np.max(input_data_file_t)}")
print(f"Min value in Input.npy: {np.min(input_data_file_t)}")


# the numbers should match

Shape of combined spectrogram array with batch and channel dimensions: (2328, 96, 128)
Number of files processed: 2328
Shape of combined spectrogram array Input.npy: (2328, 96, 128)
Number of files processed: 2328
Max value in Input.npy: 11.04010009765625
Min value in Input.npy: 0.0


In [18]:
# display first five rows in the all_spectograms
print(all_spectrograms[:5])

[[[6.56488562e+00 6.54316187e+00 6.46434736e+00 ... 5.83979464e+00
   5.78715324e+00 5.62480450e+00]
  [5.79844141e+00 1.78929567e-01 2.01793462e-02 ... 1.39120389e-02
   7.30911568e-02 4.55659674e-03]
  [9.27449614e-02 2.31717467e-01 4.87126559e-02 ... 3.32325920e-02
   1.10979183e-02 6.13076054e-02]
  ...
  [3.16771257e-05 6.59156824e-04 2.27157972e-04 ... 4.34516609e-04
   2.52580736e-04 4.66992497e-05]
  [2.05123841e-04 1.06855303e-04 8.53602978e-05 ... 2.45186948e-06
   4.36563656e-04 5.76548482e-05]
  [7.11280387e-04 1.62877215e-04 5.42714726e-04 ... 1.80369461e-05
   1.01560319e-03 1.57285773e-04]]

 [[5.32498074e+00 5.53522110e+00 5.11152506e+00 ... 3.88275313e+00
   3.18697906e+00 4.12757158e+00]
  [3.34692526e+00 1.27646960e-02 9.14698020e-02 ... 6.19779408e-01
   4.86656904e-01 1.04293919e+00]
  [7.27228165e-01 8.87380838e-01 1.21420823e-01 ... 1.46332726e-01
   1.51902914e-01 3.25587727e-02]
  ...
  [2.89123110e-03 1.64264180e-02 7.50789535e-04 ... 5.43438736e-03
   4.35965